#### Cemetery check pictures in SPA och söka i SPA
* [This notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20cemetery.ipynb) 


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-01-03 05:25:45.846070


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/search.php"  


In [3]:
# wd:Q26257009. Gamla Gävle Behravningsplats
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
# wd:Q26257009. Gamla Gävle Begravningsplats
# Q252312


queryGefle = """SELECT (CONCAT (?itemLabel," ",str(year(?birthDate))) AS ?search) ?item ?itemLabel 
  (str(year(?birthDate)) AS ?year)
  WHERE {
  ?item wdt:P119 wd:Q26257009. 
  minus {?item wdt:P4819 ?c }
  OPTIONAL { ?item wdt:P569 ?birthDate. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
} order by ?itemLabel """

queryNoPicNorra1890 = """SELECT (CONCAT (?itemLabel," ",str(year(?birthDate))) AS ?search) ?item ?itemLabel 
  (str(year(?birthDate)) AS ?year)
  WHERE {
  ?item wdt:P119 wd:Q252312. 
  minus {?item wdt:P4819 ?c }
  minus {?item wdt:P18 ?d }
  OPTIONAL { ?item wdt:P569 ?birthDate. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
  FILTER ( YEAR(?birthDate) < 1880)
} order by ?itemLabel """

 
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

#SPAdf = get_sparql_dataframe(endpoint_url, query)
#SPAdf = get_sparql_dataframe(endpoint_url, queryNoPicNorra1890)
SPAdf = get_sparql_dataframe(endpoint_url, queryGefle)

SPAdf.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   search     74 non-null     object
 1   item       75 non-null     object
 2   itemLabel  75 non-null     object
 3   year       74 non-null     object
dtypes: object(4)
memory usage: 2.5+ KB


In [4]:
  SPAdf

,search,item,itemLabel,year
0,Abraham Nordström 1796,http://www.wikidata.org/entity/Q88637598,Abraham Nordström,1796
1,Abraham Wilhelm Nordström 1828,http://www.wikidata.org/entity/Q88653430,Abraham Wilhelm Nordström,1828
2,Aina Wallberg-Ewerlöf 1890,http://www.wikidata.org/entity/Q46400003,Aina Wallberg-Ewerlöf,1890
3,Anna Lovisa Larsson 1864,http://www.wikidata.org/entity/Q87106605,Anna Lovisa Larsson,1864
4,Anna Maria Forssén 1870,http://www.wikidata.org/entity/Q88770303,Anna Maria Forssén,1870
...,...,...,...,...
70,Stig Gavlén 1927,http://www.wikidata.org/entity/Q88456505,Stig Gavlén,1927
71,Thekla Daniella Engelmark 1847,http://www.wikidata.org/entity/Q88249448,Thekla Daniella Engelmark,1847
72,Theresia Charlotta Lönnerberg 1841,http://www.wikidata.org/entity/Q89006806,Theresia Charlotta Lönnerberg,1841
73,Tommy Sandlin 1944,http://www.wikidata.org/entity/Q344440,Tommy Sandlin,1944


In [5]:
SPAdetail = "https://portrattarkiv.se/details/"
for index, row  in SPAdf.iterrows():
    print("\n",row["search"],row["item"])
    encoded_body = json.dumps({
        "limit": "5",
        "from": "0",
        "birthyear":row["year"],
        "all":row["search"]
    })
    
    r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

    if r.status != 200:
        print(r.status)
        continue
    
    data = json.loads(r.data.decode('utf-8'),) 
    urls = []


    for h in data["hits"]["hits"]:
        id = h["_id"]
        #print(h)
        source = h["_source"]
        try:
 #           urlPicture = urlbasePic + id
 #           urls.append(urlPicture)
            score = h["_score"]
            FirstName = source["FirstName"]
            LastName = source["LastName"]
            BirthYear = source["BirthYear"]
            print("\t\t",score,FirstName, " ", LastName, " - ", BirthYear,SPAdetail+id, "\t", )
        except:
            print("Error")



 Abraham Nordström 1796 http://www.wikidata.org/entity/Q88637598
		 17.82449 Sten   Leijonhufvud  -  1796 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABZeNw 	
		 15.528971 Olof   Fåhræus  -  1796 https://portrattarkiv.se/details/IQojCnw0WmAAAAAAAABKvA 	
		 15.369695 Bernhard   von Beskow  -  1796 https://portrattarkiv.se/details/5TJc-sPXaKAAAAAAAAAntA 	
		 15.296571 Axel   Belfrage  -  1796 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABKjEg 	
		 15.236661 Olof   Fåhraeus  -  1796 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABZLeQ 	

 Abraham Wilhelm Nordström 1828 http://www.wikidata.org/entity/Q88653430
		 23.027477 Abraham Viktor   Rydberg  -  1828 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB4vjQ 	
		 22.072563 Abraham   Rydberg  -  1828 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAQrQ 	
		 20.714912 Viktor   Rydberg  -  1828 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACd0Cw 	
		 20.529308 Viktor   Rydberg  -  1828 https://portrattarkiv.se/details/

		 23.939152 Per   Björkman  -  1825 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAbNg 	
		 23.913141 Per   Björkman  -  1825 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAbNQ 	
		 23.825289 Per   Björkman  -  1825 https://portrattarkiv.se/details/TNQaWo324IAAAAAAAADw6A 	
		 23.68623 Per   Björkman  -  1825 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAbNw 	
		 23.433294 Per   Björkman  -  1825 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAbNA 	

 Carin Lindskog 1886 http://www.wikidata.org/entity/Q16595675
		 18.963669 Oscar   Lindskog  -  1886 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAABicg 	
		 17.279644 Oscar   Lindskog  -  1886 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAABicw 	
		 14.981524 Catharina   Pihl  -  1886 https://portrattarkiv.se/details/Dysd0OxEWfAAAAAAAABkCg 	
		 14.798911 Catharina   Pihl  -  1886 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABTblQ 	
		 14.398691 Thorfrid   Anners  -  1886 https://portrattarkiv.se/details/sj

		 18.521774 Karl   Danelius  -  1824 https://portrattarkiv.se/details/_P7S9YzQCjAAAAAAAAAqnA 	
		 18.448074 Karl   Danelius  -  1824 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABLXoA 	
		 18.156107 Gustaf   von Schoultz  -  1824 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABMKYQ 	
		 17.91662 Carl   Kruuse-af-Verchou  -  1824 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAvDg 	
		 17.562157 Johan   Pettersson  -  1824 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABL9Hw 	

 Fredrik Wilhelm Petre 1852 http://www.wikidata.org/entity/Q88372106
		 22.356655 Fredrik   Svenonius  -  1852 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAABlCg 	
		 22.116997 Fredrik   Pegelow  -  1852 https://portrattarkiv.se/details/Dysd0OxEWfAAAAAAAAAp9g 	
		 21.221653 Fredrik   Pegelow  -  1852 https://portrattarkiv.se/details/Dysd0OxEWfAAAAAAAAAp9w 	
		 20.586834 F V   Pegelow  -  1852 https://portrattarkiv.se/details/Dysd0OxEWfAAAAAAAABjRg 	
		 20.556671 Fredrik   Pegelow  -  1852 htt

		 19.830044 Vilhelmina   Arenander  -  1848 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABKEkw 	
		 17.639885 Kristina   Nilsson  -  1848 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAAAXhQ 	
		 17.639885 Kristina   Nilsson  -  1848 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAAAXhA 	
		 17.473293 Kristina   Elf  -  1848 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB4pug 	
		 17.154192 Kristina   Olofsdotter-Elf  -  1848 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB4rJg 	

 Lars Augustin Bång 1806 http://www.wikidata.org/entity/Q17154040
		 19.898209 L   Enlund  -  1806 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAAA9mQ 	
		 19.154978 Lars   Flygare  -  1806 https://portrattarkiv.se/details/IQojCnw0WmAAAAAAAAALqA 	
		 18.658697 L J   Sandahl  -  1806 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABHRDg 	
		 17.026943 Pehr   Lundberg  -  1806 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAABAqg 	
		 15.757759 Johan   Grewell  -  1806 https://portratt

		 15.352326 Robert   Wadensten  -  1902 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABg8QQ 	
		 15.270521 Robert   Wadensten  -  1902 https://portrattarkiv.se/details/ciLMGSqrYjAAAAAAAABmuw 	
		 15.183651 Robert   Carlson  -  1902 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABKYBA 	
		 15.16989 Robert   Düser  -  1902 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA8Gdw 	
		 15.147291 Robert   Christensson  -  1902 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA7ntQ 	

 Selma Matilda Nordendal 1846 http://www.wikidata.org/entity/Q88771213
		 19.914196 Selma   Uddén  -  1846 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACaEbw 	
		 18.81398 Selma   Höök  -  1846 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAABBVw 	
		 18.81398 Selma   Gerle  -  1846 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAA_Og 	
		 17.250887 Maria   Olsén  -  1846 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAR-g 	
		 17.12239 Carl   Peterson-Tretow  -  1846 https://portrattarki